# Library Import


In [21]:
import wave
import numpy as np
import matplotlib.pyplot as plt
from Crypto.Hash import SHA3_256
from Crypto.Cipher import ChaCha20_Poly1305
from Crypto.Random import get_random_bytes

# SHA-3 Digest


In [22]:
aliceKey = get_random_bytes(500)
print("Alice's Key: ", aliceKey)
hashObject = SHA3_256.new()
hashObject.update(aliceKey)
key = hashObject.digest()
print("Hash of Alice's key: ", key)

Alice's Key:  b'\x19\x05?\xa3H\xa3~\x9d\xa0\xd9\x8d\xe2\xcd\xe6Jbd\x85\x1d\x90\x01\xc4(\xaf4\xf1\rL6\x19PWK\x95RV\x81B%jkKN\xde\x13\xffD<P\xdcq\xed\x86\xe6\x12\x9b\xc2BT) r5r\x0f\xe7\x95j\xb2\x1e 7\xcd\x00D\xa9?\xdb^q\xd2\xa5\x07\xe3\xb6\x94\x18\x92<\x17\x16\xbe2\xc6\x0b!\xfbp\xbc\xc5\x04`\xfc\xd4\x8f\xb3\xf7ry\xd6&]lg\xaf\xba\xce\x83\x8b\x944K\x19\t\x86\x1bw\xb6cx\x80\xed^\xf8zm\xab\xbe\x86<F\xbdEL\xb2uv\xeb\x16\x18Z:\x1br\xb2\x8bW\x9e\xfa\xc9P\x8b\x98\xc0S\x9eP\xc3\x97\xfc\x15>\x80\xeef\xf5\x0f\xc7^\xb0?\x18\x8a-\x0c\x8a\x9a\xd51\xf5$D\xcf\x02"g\x9d\x15\x98\x13rOc.=\xc0\x91 \x96\xceo\xf7j"\x94\xfb8\x15\x94\xc9`\xea\xd5\xeaw\x19\xd4\x1c\x98\xc8\x08Y*\xcb\xfe\xf7\x1fg\x12w\x9c\xebh\x91\xa5\xdcv\xac"u@\x06\x18\xa0H\x94\xb2\xffy1\x83\x01\x84\xd9\xe2)1\xb9\x1eA\x8e\x86\xd2\xa4\xd1pa\xa8\xb3P\xe8\xc53\x0f\x1b-\xedR\xc2\x96O\xcc\x96\x91\x07\x8d\xed\x84mVc0\xaa\xd2K\x80\\\x14\x82\x8a\xcaZ\x16T5*qX\xa3\x8c(\x00\xf6y\xe3\x10x\xf0V\x08\x85M#LY}U_9\xeb\xffl\x1e\xff\x95\x0bh\xbe \xf1"p\xea\xf4\x9

# Steganography


## Function Definition


### Embedding


#### Audio Clipping Function


In [23]:
def clip_audio(audio_data, min_value=-32768, max_value=32767):
    return np.clip(audio_data, min_value, max_value)

#### Audio Embedding Function


In [24]:
def embed_audio(carrier_path, secret_path, output_path, num_bits=8):
    # Open the carrier and secret audio files
    carrier = wave.open(carrier_path, 'rb')
    secret = wave.open(secret_path, 'rb')
    # Ensure compatibility between carrier and secret audio
    if carrier.getnchannels() != secret.getnchannels() or carrier.getsampwidth() != secret.getsampwidth():
        raise ValueError(
            "Carrier and secret audio must have the same number of channels and sample width.")
    # Read audio data
    carrier_frames = np.frombuffer(carrier.readframes(
        carrier.getnframes()), dtype=np.int16)
    secret_frames = np.frombuffer(secret.readframes(
        secret.getnframes()), dtype=np.int16)
    # Ensure carrier has enough capacity to store secret
    max_secret_length = len(carrier_frames) // (16 // num_bits)
    if len(secret_frames) > max_secret_length:
        raise ValueError(f"Carrier audio cannot hold the secret audio. Max capacity: {
                         max_secret_length}, got: {len(secret_frames)}.")
    # Resize the secret frames to match embedding capacity
    padded_secret_frames = np.zeros(max_secret_length, dtype=np.int16)
    padded_secret_frames[:len(secret_frames)] = secret_frames
    # Embed the secret audio into the carrier
    embedded_frames = np.copy(carrier_frames)
    mask = (1 << num_bits) - 1  # Mask to isolate the LSBs
    embedded_frames &= ~mask  # Clear the least significant bits
    embedded_frames[:len(padded_secret_frames)] += (padded_secret_frames >>
                                                    # Embed secret bits
                                                    (16 - num_bits)) & mask
    # Clip audio values to valid range
    embedded_frames = clip_audio(embedded_frames)
    # Write the embedded audio to a new file
    embedded_audio = wave.open(output_path, 'wb')
    embedded_audio.setparams(carrier.getparams())
    embedded_audio.writeframes(embedded_frames.astype(np.int16).tobytes())
    # Close files
    carrier.close()
    secret.close()
    embedded_audio.close()
    print(f"Embedding complete. Saved to {output_path}")

### Extraction


In [25]:
def extract_audio(embedded_path, secret_output_path, secret_length, num_bits=8):
    # Open the embedded audio file
    embedded = wave.open(embedded_path, 'rb')
    # Read audio data
    embedded_frames = np.frombuffer(embedded.readframes(
        embedded.getnframes()), dtype=np.int16)
    # Extract the secret audio
    mask = (1 << num_bits) - 1  # Mask to isolate the embedded bits
    extracted_bits = (embedded_frames & mask) << (16 - num_bits)
    secret_frames = extracted_bits[:secret_length*2]
    # Apply a low-pass filter to reduce noise
    secret_frames = np.convolve(secret_frames, np.ones(
        5)/5, mode='same')  # Simple smoothing (moving average)
    # Clip audio values to valid range
    secret_frames = clip_audio(secret_frames)
    # Write the secret audio to a new file
    secret_audio = wave.open(secret_output_path, 'wb')
    secret_audio.setparams(embedded.getparams())
    secret_audio.setnframes(secret_length)
    secret_audio.writeframes(secret_frames.astype(np.int16).tobytes())
    # Close files
    embedded.close()
    secret_audio.close()
    print(f"Extraction complete. Secret audio saved to {secret_output_path}")

## Implementation


### Input


In [26]:
# Carrier audio
carrier_audio = "cover.wav"
secret_audio = "secret.wav"  # Secret audio
embedded_audio = "embedded.wav"  # Output for the embedded audio

### Embed


In [27]:
embed_audio(carrier_audio, secret_audio, embedded_audio, num_bits=8)

Embedding complete. Saved to embedded.wav


# Encryption, Decryption, MAC Generation and Authentication

## Function Definitions

Encryption using the SHA3-256 digest of the final Ekert key.

In [28]:
def encryptAudio(key, inputFile, outputFile):
    with open(inputFile, 'rb') as inputFile:
        audioData = inputFile.read()
    header = b"Stego Audio"
    cipher = ChaCha20_Poly1305.new(key=key)
    cipher.update(header)
    nonce = cipher.nonce
    print("Nonce: ", nonce.hex())
    ciphertext, tag = cipher.encrypt_and_digest(audioData)
    print("Tag: ", tag.hex())
    with open(outputFile, 'wb') as outputFile:
        outputFile.write(nonce+ciphertext+tag)

Applying random bit flip to encrypted audio in order to simulate middle-man attack

In [29]:
def addNoise(inputFile, noisyOutputFile):
    with open(inputFile, 'rb') as inputFile:
        noisyCiphertextBytes = bytearray(inputFile.read())
        for x in range(1000):
            indexToModify = np.random.randint(0, len(noisyCiphertextBytes)-1)
            noisyCiphertextBytes[indexToModify] ^= np.random.randint(
                0x00, 0xFF)
        with open(noisyOutputFile, 'wb') as noisyOutputFile:
            noisyOutputFile.write(noisyCiphertextBytes)

Decryption

In [30]:
def decryptAudio(key, inputFile, outputFile):
    with open(inputFile, 'rb') as inputFile:
        data = inputFile.read()
        nonce = data[:12]
        ciphertext = data[12:-16]
        tag = data[-16:]
    header = b"Stego Audio"
    cipher = ChaCha20_Poly1305.new(key=key, nonce=nonce)
    cipher.update(header)
    try:
        decryptedAudio = cipher.decrypt_and_verify(ciphertext, tag)
        with open(outputFile, 'wb') as outputFile:
            outputFile.write(decryptedAudio)
            print("Nonce: ", nonce.hex())
            print("Tag: ", tag.hex())
            print("Decryption and verification successful!")
    except ValueError:
        print("Verification failed!")

## Implementation

In [31]:
inputAudio = "embedded.wav"
output = "encrypted.bin"
decryptedAudio = "decrypted.wav"

In [32]:
encryptAudio(key, inputAudio, output)

Nonce:  d5d553953532bc2f38ea6c5f
Tag:  056e8dd80b50fab96725b71a7cc2902f


In [33]:
addNoise(output, "noisy.bin")
noisyOutput = "noisy.bin"

In [34]:
decryptAudio(key, output, decryptedAudio)

Nonce:  d5d553953532bc2f38ea6c5f
Tag:  056e8dd80b50fab96725b71a7cc2902f
Decryption and verification successful!


In [35]:
decryptAudio(key, noisyOutput, "outfile.bin")

Verification failed!


# Extraction

In [36]:
extracted_audio = "retrieved_audio1.wav"
# extract_audio('embedded.wav', 'output.wav', 8)

secret_audio_length = wave.open(
    secret_audio, 'rb').getnframes()  # Length of the secret audio
extract_audio(embedded_audio, extracted_audio, secret_audio_length)

Extraction complete. Secret audio saved to retrieved_audio1.wav
